In [ ]:
Following Example shows how financial derivative valuation technique can be effectively used for the valuation of  large scale investment projects which can be upfront capital outlay or in stages.

In 2003 Tesla Motors Inc. was founded by Martin Eberhard en Marc Tarpenning. A year later Elon Musk decided to invest a large amount of money to start the development of the "Tesla Roadster".

One of the major car manufacturers has also become interested in investing in electric cars. They are still in the early stages of developing a fully electric car and management is a bit skeptic about the profitability of the investment. The R&D program involved multiple stages and required immediate capital outlays, while the cash inflows would be expected at a later point in time, as shown in the figure below. Management uses an opportunity cost of capital of 16% and a risk-free rate of 2%. Cash inflows are discounted with the cost of capital and cash outflows are discounted with the risk-free rate.

The management of the company is having doubts whether they should invest in the development of the new electric car. However, you mention to them that they do not have to commit to both stages immediately. Instead they could invest in the R&D stage to create the option to commercialize later when the circumstances are right (i.e. they have the option to invest in the commercial project).

Suppose that the (gross) value of expected future cash inflows (V) may fluctuate in line with the random fluctuation in demand (u = 1.4, d = 0.7 per period). Also assume a risk-neutral probability of p = 0.457.

Cash Flows for the year are as follow:
Year 0=-20
Year 1=-100
Year 2=-1600
Year 3=400
Year 4=800
Year 5=1200
Year 6=300

In [31]:
import numpy as np 
import pandas as pd
data=pd.read_csv('CFData1.csv',index_col='T')
data=data.rename(columns={'DiscountedCash Flow':'DiscountedCashFlow'})
print(data)

   CashFlow  DiscountedCashFlow
T                              
0       -20          -20.000000
1      -100          -97.087379
2     -1600        -1537.870050
3       400          256.263069
4       800          441.832878
5      1200          571.335619
6       300          123.132676


In [15]:
NPV=np.sum(data['DiscountedCashFlow'])
print(NPV)
NPV_at_time2=np.sum(data.loc[3:,['DiscountedCashFlow']])
print(NPV_at_time)

-262.3931860400001
DiscountedCashFlow    1392.564243
dtype: float64


The expected cash flows from the project are discounted at the rate of borrowing/cost of debt whereas the initial investments in the RnD phase are discounted at the risk free rate.
It is found that the Net Present Value of the project is negative as thus,the management would deter from pursuing the project.However further analysis incorporating the flexibilties or options inherent in the project, the project might appear to be promising to invest in.

Below binomaial tree in simulated to find the value of project at the time of making the investment i.e, at time  2years when the company has to decide whether to invest in project or not.

In [22]:
def binomialTree(option_type,initialPrice,strikePrice,annualizedRiskFreeRate,u,d,timeToMaturity,numberOfSteps,
                 american="false"):
    
    # calculate small time steps for tree. 
    deltaT = float(timeToMaturity)/numberOfSteps
    
    u = 1.4
    d = 0.7
    
    #As we are going to work using vectors, we need to initiate the arrays.
    startingArray = np.asarray([0.0 for i in range(numberOfSteps+1)])
    
    #We need  tree for calculating expiration values, hence initiate that in an Array. 
    PriceArray = np.asarray([(initialPrice*(u**j)*(d**(numberOfSteps-j))) for j in range(numberOfSteps+1)])
    r = np.divide(PriceArray, initialPrice)
    
    
    #Similarly we need the tree for strike price vectorization. 
    strikeVectorization = np.asarray([float(strikePrice) for i in range(numberOfSteps+1)])
    
    #Calculate the probability using the formula
    firstValueforProbability = np.exp(annualizedRiskFreeRate*deltaT)
    p =(firstValueforProbability-d)/(u-d)
    q = 1.0 - p
    
    #Compute the different prices across all paths of the tree. 
    if option_type == "C":
        startingArray[:] = np.maximum(PriceArray-strikeVectorization,0.0)
    else:
        startingArray[:] = np.maximum(-PriceArray+strikeVectorization,0.0)
        
    #Compute backward option prices
    for i in range(numberOfSteps-1,-1,-1):
        startingArray[:-1]=np.exp(-annualizedRiskFreeRate*deltaT)*(p*startingArray[1:]+q*startingArray[0:-1])
        PriceArray[:]=PriceArray[:]*u
        
        #Check for option being American or European option and price accordingly. 
        if american == "true":
            if option_type == "C":
                startingArray[:] = np.maximum(PriceArray[:]-strikeVectorization[:],startingArray[:])
            else:
                startingArray[:] = np.maximum(-PriceArray[:]+strikeVectorization[:],startingArray[:])
                
    return startingArray[0]



value_of_project_at_time2 = binomialTree("C",1392,1600,0.02,1.4,0.7,2.0,2,"false")
print(value_of_project_at_time2)

    
    
    



226.83534020543212


Accounting for expenditures incurred between year 0 and year 2 for instance the RnD expenditures and substracting the same from the above obtained proect value to get the total project value


In [48]:
PV_of_expenditures_upto_yearTwo=-np.sum(data.loc[:1,['DiscountedCashFlow']])
print(PV_of_expenditures_upto_yearTwo)


DiscountedCashFlow    117.087379
dtype: float64


In [51]:
total_project_value=np.max(value_of_project_at_time2-PV_of_expenditures_upto_yearTwo,0)
print(total_project_value)

109.74796156543212


In [52]:
value_of_option_to_invest=total_project_value-NPV
print(value_of_option_to_invest)

372.14114760543225


Thus it is found that the value of the option to invest in the project or not is less than zero  and hence it prudent to take up the project which otherwise not possible to  infer from the result of traditional DCF analysis.